## export to shapefiles

export data that is stored in the devinfo access database to shapefiles. the file name of each shapefile will be its layer id as defined in the access database 

As we are using the **[ArcPy](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm)** python library, we will need to start our Jupyter Notebook the following way:

- Open ArcGIS Pro
- Click the **Project** tab and click **Python** to access the **Python Package Manager**. To create, edit or remove environments click on the **Manage Environments** button.
- Clone the existing default python environment as the default is read-only. *[reference](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-conda.htm#ESRI_SECTION2_1B154688AEE0497F836B3FFBAAAD0C8C)*
- Find the `pyodbc` package in the list and install it. *[reference](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-conda.htm#ESRI_SECTION2_85BC919097434B3B9AE1A746D793AA29)*
- **IMPORTANT** Open the `Python Command Prompt` and verify the path is to your new, cloned enviornment
- Use the `cd <path>` command to change to your working directory
- From the `Python Command Prompt`, start your Jupyter Notebook server using the `jupyter notebook` command

In [ ]:
import os
import arcpy
import pyodbc
# [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]
# if you see [], you may need to install the 64-bit ACE drivers
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
# 64bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255

from IPython.display import clear_output

In [ ]:
access_database = r'Z:\dev\nepal-import\NepalInfo2016_for_python.accdb'
output_base = 'nepal'
output_shapes_folder = 'shapes'

In [ ]:
connStr = (
    r'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};'
    r'DBQ={};'.format(access_database)
)

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT layer_shp, layer_shx, layer_dbf, layer_nid, layer_name, area_id, area_level, UT_Area_en.area_parent_nid
FROM export_shps 
ORDER BY area_level ASC
"""
crsr = cnxn.execute(sql)

# export just a subset for testing
# rows = crsr.fetchmany(50)

# uncomment this to get crazy and export them all!!
rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

## Create Shapefiles

In [ ]:
row_count = len(rows)

layer_checks = []

for i, row in enumerate(rows):
    if i == 5:
        break

    shp = row[0]
    shx = row[1]
    dbf = row[2]

    layer_id = row[3]

    if layer_id in layer_checks:
        continue

    layer_checks.append(layer_id)

    out_file = '{}/{}'.format(os.path.join(output_base, output_shapes_folder), layer_id)

    print ('adding {} of {}'.format(i, row_count))
    clear_output(wait=True)

    with open('{}.shp'.format(out_file), 'wb') as writer:
        writer.write(shp)
    with open('{}.shx'.format(out_file), 'wb') as writer:
        writer.write(shx)
    with open('{}.dbf'.format(out_file), 'wb') as writer:
        writer.write(dbf)
    with open('{}.prj'.format(out_file), 'w') as writer:
        writer.write('GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]')